In [1]:
visible_seqs = []  # 观测序列
hidden_seqs = []  # 状态序列
char2idx = {}
# {B,M,E,S}标注集,即词首(Begin),词尾(End),词中(Middle)以及单字成词(Single)
idx2hidden = {0: "B", 1: "M", 2: "E", 3: "S"}

In [2]:
count = 0
for line in open("data.txt", encoding="utf8"):
    visible_seq = []
    hidden_seq = []
    arrs = line.strip().split(" ")
    for item in arrs:
        if len(item) == 1:
            hidden_seq.append(3)
        elif len(item) == 2:
            hidden_seq.extend([0, 2])
        else:
            hidden_seq.extend([0] + [1] * (len(item) - 2) + [2])

        for c in item:
            if c in char2idx:
                visible_seq.append(char2idx[c])
            else:
                char2idx[c] = count
                visible_seq.append(count)
                count += 1

        visible_seqs.append(visible_seq)
        hidden_seqs.append(hidden_seq)

In [3]:
len(char2idx)  # 观测序列共有4656个

4656

In [4]:
from MyHMM import MyHMM

hmm = MyHMM(hidden_status_num=4, visible_status_num=len(char2idx))
hmm.supervision(visible_seqs, hidden_seqs)  # 已有已经标注好的训练数据

In [5]:
def seg(vis, hid):
    """
    根据状态序列拆解句子
    :param vis:要拆解的句子
    :param hid:状态序列
    :return:拆解后的句子
    """
    rst = []
    for i in range(0, len(hid)):
        if hid[i] in [2, 3]:
            rst.append(vis[i])
            rst.append("/")
        else:
            rst.append(vis[i])
    return "".join(rst)

In [6]:
example0 = "我和我的祖国，一刻也不能分离"
seg(example0, hmm.viterbi([char2idx[c] for c in example0])[0])

'我/和/我/的/祖国/，/一刻/也/不能/分离/'

In [7]:
example1 = "小龙女说，我也想过过过过过过过的生活"
seg(example1, hmm.viterbi([char2idx[c] for c in example1])[0])

'小龙/女/说/，/我/也/想过/过过/过过/过过/的/生活/'

In [8]:
example2 = "萌萌你有在学校好好学习吗，没有"
seg(example2, hmm.viterbi([char2idx[c] for c in example2])[0])

'萌萌/你/有/在/学校/好/好/学习/吗/，/没有/'